In [1]:
from datetime import datetime
start = datetime.now()

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import io
import csv
from cx_Oracle import DatabaseError
import os
import time

address = ''
engine = create_engine(address)

In [2]:
## Needs to be fixed ###############################################################################################
cols = ['dbuid', 'dbpop2016', 'dbtdwell2016', 'dburdwell2016', 'dbarea2016', 'dbir2016', 'dauid', 'darplamx', 'darplamy', 
         'darplat', 'darplong','pruid', 'prname', 'prename', 'prfname', 'preabbr', 'prfabbr']

#,'feduid', 'fedname', 'eruid','ername', 'cduid', 'cdname', 'cdtype', 'adauid', 'adacode', 'csduid', 'csdname', 'csdtype', 
# 'sactype', 'saccode',
#'ccsuid', 'ccsname', 'dpluid', 'dplname', 'dpltype', 'cmapuid', 'cmauid', 'cmaname', 'cmatype', 'ctuid', 'ctcode',
#'ctname', 'popcrrapuid', 'popctrrauid', 'popctrraname', 'popctrratype', 'popctrraclass']
####################################################################################################################
#Load source data to pandas dataframe *Not strictly necessary for bulk loading
df = pd.read_csv(r'PopulationAndDwellings.csv',encoding='latin_1', names=cols, header=0, dtype='str', 
                    usecols=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])

In [3]:
# Create Oracle connection and cursor
connection = engine.raw_connection()
cursor = connection.cursor()

#create the table but first drop if it already exists
try:
    command = """DROP TABLE src_statscan_popdwell""" #Need to figure out how to check if table exists before attempting to drop
    cursor.execute(command)
except DatabaseError as e:
    print(e)
    
command = """
    CREATE TABLE src_statscan_popdwell( 
      dbuid VARCHAR2(11), 
      dbpop2016 INTEGER, 
      dbtdwell2016 INTEGER, 
      dburdwell2016 INTEGER, 
      dbarea2016 NUMBER(12,4), 
      dbir2016 char(1), 
      dauid VARCHAR2(8), 
      darplamx NUMBER(17,8), 
      darplamy NUMBER(17,8), 
      darplat NUMBER(9,6), 
      darplong NUMBER(11,6), 
      pruid VARCHAR2(2), 
      prname VARCHAR2(100), 
      prename VARCHAR2(100), 
      prfname VARCHAR2(100),
      preabbr VARCHAR2(10),
      prfabbr VARCHAR2(10),
      feduid VARCHAR2(5),
      fedname VARCHAR2(200),
      eruid VARCHAR2(4),
      ername VARCHAR2(100),
      cduid VARCHAR2(4),
      cdname VARCHAR2(100),
      cdtype VARCHAR2(3),
      adauid VARCHAR2(8),
      adacode VARCHAR2(4),
      csduid VARCHAR2(7),
      csdname VARCHAR2(100),
      csdtype VARCHAR2(3),
      sactype VARCHAR2(1),
      saccode VARCHAR2(3),
      ccsuid VARCHAR2(7),
      ccsname VARCHAR2(100),
      dpluid VARCHAR2(6),
      dplname VARCHAR2(100),
      dpltype VARCHAR2(3),
      cmapuid VARCHAR2(5),
      cmauid VARCHAR2(3),
      cmaname VARCHAR2(110),
      cmatype VARCHAR2(1),
      ctuid VARCHAR2(10),
      ctcode VARCHAR2(4),
      ctname VARCHAR2(7),
      popcrrapuid VARCHAR2(6),
      popctrrauid VARCHAR2(4),
      popctrraname VARCHAR2(100),
      popctrratype VARCHAR2(1),
      popctrraclass VARCHAR2(1),
      CONSTRAINT carto_db_pkey PRIMARY KEY(dbuid)
)"""

cursor.execute(command)
connection.commit()
#close cursor and connection
cursor.close()
connection.close()

#stream the data from pandas using 'to_csv' and StringIO(); then use cx_oracles 'execute_many' function
df_list = np.array_split(df,50)
frame_count=len(df_list)
i = 0

for frame in df_list:
    i += 1
    output = io.StringIO()
    frame.to_csv(output, sep=',', encoding='latin_1', index=False)
    output.seek(0)
    reader = csv.DictReader(output)
    rows = [row for row in reader]
        
    command = """
        INSERT INTO 
            src_statscan_popdwell( 
                dbuid, dbpop2016, dbtdwell2016, dburdwell2016, dbarea2016, dbir2016, dauid, darplamx, darplamy, darplat, darplong,
                pruid, prname, prename, prfname, preabbr, prfabbr)
        VALUES (
                :dbuid, :dbpop2016, :dbtdwell2016, :dburdwell2016, :dbarea2016, :dbir2016, :dauid, :darplamx, :darplamy, :darplat, 
                :darplong, :pruid, :prname, :prename, :prfname, :preabbr, :prfabbr)
    """
#, feduid, fedname, eruid, ername, cduid, cdname, cdtype, adauid, 
#  adacode, csduid, csdname, csdtype, sactype, saccode, ccsuid, ccsname, dpluid, dplname, dpltype, cmapuid, cmauid, 
#  cmaname, cmatype, ctuid, ctcode, ctname, popcrrapuid, popctrrauid, popctrraname, popctrratype, popctrraclass
    
#, :feduid, :fedname, :eruid:ername, :cduid, 
#  :cdname, :cdtype, :adauid, :adacode, :csduid, :csdname, :csdtype, :sactype, :saccode, :ccsuid, :ccsname, :dpluid,
#  :dplname, :dpltype, :cmapuid, :cmauid, :cmaname, :cmatype, :ctuid, :ctcode, :ctname, :popcrrapuid, :popctrrauid,
#  :popctrraname, :popctrratype, :popctrraclass

    commited = False
    while commited == False:
        try:
            # Create Oracle connection and cursor
            connection = engine.raw_connection()
            cursor = connection.cursor()
            cursor.prepare(command)
            cursor.executemany(None, rows)
            connection.commit()
            commited = True
        except Exception as e:
            print(e)
            print('Trying Again in 10 seconds')
            time.sleep(10)
            
                       
    print('Committed frame {0} of {1}'.format(i,frame_count))

        
    #close cursor and connection
    cursor.close()
    connection.close()
print('Finished')

Committed frame 1 of 50
Committed frame 2 of 50
Committed frame 3 of 50
Committed frame 4 of 50
Committed frame 5 of 50
Committed frame 6 of 50
Committed frame 7 of 50
Committed frame 8 of 50
Committed frame 9 of 50
Committed frame 10 of 50
Committed frame 11 of 50
Committed frame 12 of 50
Committed frame 13 of 50
Committed frame 14 of 50
Committed frame 15 of 50
Committed frame 16 of 50
Committed frame 17 of 50
Committed frame 18 of 50
Committed frame 19 of 50
Committed frame 20 of 50
Committed frame 21 of 50
Committed frame 22 of 50
Committed frame 23 of 50
Committed frame 24 of 50
ORA-03135: connection lost contact
Process ID: 13031
Session ID: 62 Serial number: 9953

Trying Again in 10 seconds
Committed frame 25 of 50
Committed frame 26 of 50
Committed frame 27 of 50
Committed frame 28 of 50
Committed frame 29 of 50
Committed frame 30 of 50
Committed frame 31 of 50
Committed frame 32 of 50
Committed frame 33 of 50
Committed frame 34 of 50
Committed frame 35 of 50
Committed frame 36